In [1]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

### §1. Create a Database
We first create a database with three tables: temperatures, stations, and countries. 

In [8]:
url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(url)

In [9]:
url="https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries=pd.read_csv(url)
    

In [10]:
conn = sqlite3.connect("temps.db")#create the database

We write a function to clean the data.

In [11]:
def prepare_df(df):
    '''
    Stack the temperature columns into a single column.
    '''
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)

We now populate the three tables in our database.

In [12]:
df_iter = pd.read_csv("/Users/caitlin/Documents/Math/PIC16B/week2/temps.csv", chunksize = 100000)
for df in df_iter:
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "append", index = False)

In [14]:
stations.to_sql("stations", conn, if_exists = "replace", index = False)
countries.to_sql("countries", conn, if_exists = "replace", index = False)

/Users/caitlin/opt/anaconda3/envs/PIC16B/lib/python3.7/site-packages/pandas/core/generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [15]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('temperatures',), ('stations',), ('countries',)]


As we can see, there are now three tables in our database.

### §2. Write a Query Function

We write a function that returns the temperature readings at all of the stations during the specified year and month and country.

In [94]:
def query_climate_database(country,year_begin,year_end,month):
    '''
    The return value of query_climate_database() is a Pandas dataframe of 
    temperature readings for the specified country, in the specified date range, 
    in the specified month of the year.
    '''
    conn = sqlite3.connect("temps.db")
    cmd =\
    """
    SELECT S.name,S.latitude, S.longitude, T.month, T.temp,T.year,T.id
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    WHERE T.year>=%s AND T.year<%s AND T.month=%s 
    """
    cmd2=\
    """
    SELECT C.'FIPS 10-4'
    FROM countries C
    WHERE C.name=='%s'
    """
    df = pd.read_sql_query(cmd%(year_begin,year_end,month), conn)
    cursor = conn.cursor()
    cursor.execute(cmd2%(country))
    code = cursor.fetchone()[0]
    conn.close()
    df = df[df["ID"].str[0:2]==code]
    df = df.drop(["ID"], axis = 1)
    df["Country"]=country
    return df
    

For example:

In [3]:
query_climate_database("India",1980,2020,1)

,NAME,LATITUDE,LONGITUDE,Month,Temp,Year,Country
162994,PBO_ANANTAPUR,14.583,77.633,1,23.48,1980,India
162995,PBO_ANANTAPUR,14.583,77.633,1,24.57,1981,India
162996,PBO_ANANTAPUR,14.583,77.633,1,24.19,1982,India
162997,PBO_ANANTAPUR,14.583,77.633,1,23.51,1983,India
162998,PBO_ANANTAPUR,14.583,77.633,1,24.81,1984,India
...,...,...,...,...,...,...,...
166054,DARJEELING,27.050,88.270,1,5.10,1983,India
166055,DARJEELING,27.050,88.270,1,6.90,1986,India
166056,DARJEELING,27.050,88.270,1,8.10,1994,India
166057,DARJEELING,27.050,88.270,1,5.60,1995,India


### §3. Write a Geographic Scatter Function for Yearly Temperature Increases
Question: How does the average yearly change in temperature vary within a given country?

In [98]:
from sklearn.linear_model import LinearRegression

def coef(data_group):
    '''
    Perform linear regression 
    returns the first coefficient
    '''
    x = data_group[["Year"]] 
    y = data_group["Temp"]  
    LR = LinearRegression()
    LR.fit(x, y)
    return round(LR.coef_[0],4)

In [99]:
from plotly import express as px
import calendar
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    '''
    returns a interactive geographic scatterplot with a point for each station, 
    such that the color of the point reflects an estimate of the yearly change 
    in temperature during the specified month and time period at that station.
    '''
    df = query_climate_database(country,year_begin,year_end,month)
    df["count"] = df.groupby(['NAME'])["Month"].transform('count')
    df = df[df["count"]>=min_obs]
    coefs = df.groupby(["NAME"]).apply(coef)
    coefs = coefs.reset_index()
    df = pd.merge(df, coefs, on = ["NAME"])
    df = df.drop(["Temp","Year"], axis = 1)
    df = df.drop_duplicates()
    df = df.rename(columns={0: 'Estimated Yearly Increase (°C)'})
    fig = px.scatter_mapbox(df, 
                        lat = "LATITUDE",
                        lon = "LONGITUDE", 
                        hover_name = "NAME", 
                        color = "Estimated Yearly Increase (°C)",
                        title = "Estimates of yearly increase in {} for stations in {}, years {}-{}".format(calendar.month_name[month],country,year_begin,year_end),
                        height = 300,
                        **kwargs)
    return fig

For example: we can see from the plot below that in China, from 1980 to 2020, in January, the temperature at most of the stations increases.

In [11]:
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig = temperature_coefficient_plot("China", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

Example:Unitied States

In [103]:
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig = temperature_coefficient_plot("United States", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

### §4. Create Two More Interesting Figures

### Temperature Anomaly Detection
Question: For each temperature reading, how does that reading compare to the average reading in that month and at a particular station?

We compute the z-score of each temperature reading.

In [10]:
def z_score(x):
    m = np.mean(x)
    s = np.std(x)
    return (x - m)/s

In [76]:
import plotly.express as px
import plotly.graph_objects as go
def outlier(station):
    '''
    A temperature reading with a |z-score| > 2 is categorized as a anomaly.
    In the figure, the red scatter points corresponds to points with z > 2
    the blue scatter points corresponds to points with z < -2
    '''
    conn = sqlite3.connect("temps.db")
    cmd = \
    """
    SELECT S.name, T.year, T.month, T.temp
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    WHERE S.name = '%s'
    """
    df = pd.read_sql_query(cmd%(station), conn)
    conn.close()
    df["z"] = df.groupby(["NAME", "Month"])["Temp"].transform(z_score)
    df["Date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"].astype(str))
    anomalies = df[np.abs(df["z"]) > 2]
    fig = go.Figure()
    
    color = [0 if v > 2 else 1 if v < -2 else 3 for v in anomalies["z"]]

    colorscale = [[0, 'red'],  [1, 'blue']]

    # Add traces
    fig.add_trace(go.Scatter(x=df["Date"], y=df["Temp"],
                    mode='lines',
                    marker_color = 'lightgrey',
                    name="Temperature"  )
                    )
    fig.add_trace(go.Scatter(x=anomalies["Date"], y=anomalies["Temp"],
                    mode='markers',
                    name='Anomalies',
                    marker=dict(
                    color=color,
                    colorscale=colorscale
                     )))
    
    fig.update_layout(
    title="Temperature Anomalies at {}".format(station),
    margin={"r":0,"t":30,"l":0,"b":0},
    width=1000,
    height=300,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
    fig.show()
    

For example: in Beijing, there are more anomalies that are anomalously warm appearing after 2000.

In [77]:
outlier("BEIJING")

### Standard Deviation of temperature readings in each month at a specified station
Question: at a specified station, how does the temperature in each month varies in these years? We use standard deviation to measure average amount of variability.

In [68]:
def std(station):
    conn = sqlite3.connect("temps.db")
    cmd = \
    """
    SELECT S.name, T.year, T.month, T.temp
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    WHERE S.name = '%s'
    """
    df = pd.read_sql_query(cmd%(station), conn)
    conn.close()
    df2 = df.groupby(["NAME", "Month"])["Temp"].aggregate(np.std)
    df2 = df2.reset_index()
    df2 = df2.rename(columns={"Temp": 'std'})
    
    fig = px.line(df2, 
                 x = "Month", 
                 y = "std",
                 width = 500,
                 height = 300,
                 title ="Temperature Standard Deviation at {}".format(station))
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

    fig.show()
    

We see from the plot below, the temprature during winter has a larger variation than during summer in Beijing.

In [69]:
std("BEIJING")

The temperature has a larger variation in January and February.

In [101]:
std("U_C_L_A")